In [ ]:
# credits:
# https://www.kaggle.com/code/bsmit1659/aimo-vllm-accelerated-tot-sc-deepseekmath

In [1]:
!pip uninstall -y torch -q
!pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2024.3.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
dask-cuda 23.8.0 requires pynvml<11.5,>=11.0.0, but you have pynvml 11.5.0 which is incompatible.
raft-dask 23.8.0 requires dask==2023.7.1, but you have dask 2024.3.1 which is incompatible.


In [2]:
from vllm import LLM, SamplingParams
import pandas as pd
from tqdm import tqdm
import gc
import re
import sys
import subprocess
from collections import defaultdict, Counter
import numpy as np
from transformers import (AutoModelForCausalLM,
    AutoTokenizer,
    set_seed)
import torch
import math

llm = LLM(model="/kaggle/input/deepseek-math",
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.99,
          swap_space=4,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)

tokenizer = llm.get_tokenizer()

good_token = '+'
bad_token = '-'
step_tag = 'ки'

prm_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm')
prm_candidate_tokens = prm_tokenizer.encode(f"{good_token} {bad_token}")[1:] # [648, 387]
step_tag_id = prm_tokenizer.encode(f"{step_tag}")[-1] # 12902
prm_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm',
                                                 torch_dtype=torch.float16,
                                                 device_map="balanced_low_0").eval()

2024-04-27 17:16:38,060	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 04-27 17:16:39 config.py:767] Casting torch.bfloat16 to torch.float16.
INFO 04-27 17:16:39 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-27 17:16:39 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/kaggle/input/deepseek-math', tokenizer='/kaggle/input/deepseek-math', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=fp8_e5m2, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-27 17:16:41 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-27 17:16:41 selector.py:25] Using XFormers backend.
INFO 04-27 17:18:14 model_runner.py:104] Loading model weights took 12.8725 GB
INFO 04-27 17:18:16 gpu_executor.py:94] # GPU blocks: 177, # CPU blocks: 1092


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


In [4]:
def eval_prm(candidates):
    # Initialize a list to store all the log probabilities
    all_log_probs = []
    # Process the candidates in batches
    for i in range(len(candidates)):
        # Select a batch of candidates
        input_ids = prm_tokenizer.encode(candidates[i], return_tensors="pt").to("cuda:1")  # Concatenate the padded inputs into a tensor

        with torch.no_grad():
            logits = prm_model(input_ids).logits[:, :, prm_candidate_tokens] # b,l,C
            scores = logits.softmax(dim=-1)[:, :, 0].squeeze() # l
            # Collect the log probabilities from this batch
            all_log_probs.append(scores[-1].item())
    return all_log_probs

In [ ]:
stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
stop_words.append("\n")

sampling_params = SamplingParams(temperature=1,
                                 max_tokens=256,
                                 min_tokens=32,
                                 stop=stop_words)

cot_instruction = "\nPlease reason step by step, and put your final answer within \\boxed{}."


n = 5 # beams
samples = 7
max_depth = 24
overlap_threshold = 0.6
all_prompts = []
total_paths = []
total_answers = []

def is_integer(num):
    if isinstance(num, float):
        return num.is_integer()
    elif isinstance(num, int):
        return True
    else:
        return False
    
def is_between_0_and_999(num):
    return 0 <= num <= 999

def prm_prompt(text, current_level):
    return f"Step {str(current_level)}:" + text + ' ки'

def remove_prm_prompt(text):
    pattern = r"Step \d+:"
    text = re.sub(pattern, "", text)
    return text.replace(" ки","")

import re
def extract_number(text):
    patterns = [
        r'The answer is.*\\boxed\{(.*?)\}',
        r"The answer is[:\s]*\$([0-9]+)\$",
        r"The answer is[:\s]*([0-9]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1)
    return 'parse err'

def tot_agg(completed_paths):
    # [(answer,score,len_,current_level),...]
    pass

def get_overlap(nodes_split,node_list):
    max_overlap = float("-inf")
    node_len = len(node_list)
    for previous_split in nodes_split:
        count = 0
        len_ = max(len(previous_split),node_len)
        for i,j in zip(previous_split,node_list):
            count += (i==j)
        count /= len_
        max_overlap = max(max_overlap,count)
    return max_overlap

for i in tqdm(range(len(df))):
    id_ = df['id'].loc[i]
    problem = df['problem'].loc[i]

    messages = [
        {
            "role": "user",
            "content": problem + cot_instruction
        }
    ]

    base_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    current_level = 1

    current_level_nodes = [base_prompt]
    current_scores = [float('inf')] # for min score
    completed_paths = []
    completed_path_splits = []
    
    while (len(completed_paths) < n) and (current_level < max_depth) and (current_level_nodes):
        # for generation, remove special tokens for PRM
        batch_responses = llm.generate([remove_prm_prompt(t) for t in current_level_nodes]*samples, sampling_params)
        prm_inputs = []
        cumulative_lens = []

        # Collect candidates for reward model evaluation
        for candidate,parent in zip(batch_responses,current_level_nodes*samples):
            prm_input = parent + prm_prompt(candidate.outputs[0].text,current_level)
            cumulative_tokens = len(candidate.prompt_token_ids) + len(candidate.outputs[0].token_ids)
            prm_inputs.append(prm_input)
            cumulative_lens.append(cumulative_tokens)
        # Get the indices of unique elements in prm_inputs
        unique_indices = [i for i, x in enumerate(prm_inputs) if prm_inputs.index(x) == i]
        prm_inputs = [prm_inputs[i] for i in unique_indices]
        current_scores = [(current_scores*samples)[i] for i in unique_indices]
        cumulative_lens = [cumulative_lens[i] for i in unique_indices]
        
        # Batch reward model evaluation
        prm_scores = eval_prm(prm_inputs)
        prm_scores = [min(old,new) for old,new in zip(current_scores,prm_scores)]
        next_level_nodes = []
        next_scores = []
        nodes_split = []
        
        # Prune to keep only the top 'n' candidates based on scores
        combined = list(zip(prm_inputs,prm_scores,cumulative_lens))
        combined.sort(key=lambda x: x[1], reverse=True)  # Sort nodes by their scores
        for node,score,len_ in combined:
            answer = extract_number(remove_prm_prompt(node))
            if answer == 'parse err': # not finished
                if len_ > 2048: continue # max out len_
                node_list = node.split(" ки")
                if get_overlap(nodes_split,node_list) < overlap_threshold:
                    next_level_nodes.append(node)
                    next_scores.append(score)
                    nodes_split.append(node_list)
                else: continue
                if len(next_level_nodes) == n: break
            else: # finished
                node_list = node.split(" ки")
                if get_overlap(completed_path_splits,node_list) < overlap_threshold:
                    try:
                        answer = eval(answer)
                        if is_integer(answer) and is_between_0_and_999(answer):# correct format
                            completed_paths.append((answer,score,len_,current_level))
                            completed_path_splits.append(node_list)
                    except: # bad eval
                        continue
        # if current_level_nodes is empty, all max out or err out. exit loop
        current_scores, current_level_nodes = next_scores, next_level_nodes
        current_level += 1
    
    print(f'problem {i}, sol {completed_paths}')
    total_paths.append(completed_paths)
#     total_answers.append(final_answer)

In [ ]:
completed_paths

In [7]:
len(set(current_level_nodes)),len(current_level_nodes),len(set(prm_inputs)),len(prm_inputs)

(5, 5, 32, 32)

In [1]:
total_paths = [[(28, 0.71240234375, 724, 8),
                (52, 0.658203125, 750, 11),
                (41, 0.6259765625, 822, 11),
                (28, 0.418701171875, 866, 11),
                (36, 0.385009765625, 1301, 17)],
                [(480, 0.611328125, 497, 4),
                (9, 0.59619140625, 447, 4),
                (6, 0.67919921875, 415, 5),
                (898, 0.20947265625, 687, 5),
                (6, 0.67919921875, 451, 6)],
                [(17, 0.615234375, 659, 8),
                (9, 0.515625, 816, 8),
                (5, 0.64404296875, 655, 9),
                (2, 0.56982421875, 583, 9),
                (146, 0.48046875, 746, 9)],
                [(800, 0.16455078125, 1262, 18)],
                [(310, 0.82421875, 294, 1),
                (310, 0.8154296875, 286, 3),
                (310, 0.8154296875, 293, 4),
                (310, 0.763671875, 473, 5),
                (310, 0.763671875, 617, 6),
                (310, 0.57763671875, 363, 6)],
                [(199, 0.56591796875, 386, 6),
                (199, 0.48046875, 372, 8),
                (101, 0.51171875, 490, 9),
                (197, 0.449462890625, 753, 9),
                (199, 0.42626953125, 453, 9)],
                [(194, 0.6689453125, 326, 3),
                (97, 0.49609375, 386, 3),
                (97, 0.84375, 459, 5)],
                [(100, 0.7216796875, 598, 7),
                (256, 0.69921875, 595, 8),
                (256, 0.6298828125, 660, 8),
                (256, 0.615234375, 579, 8),
                (100, 0.418701171875, 492, 9)],
                [(29, 0.52734375, 493, 5),
                (29, 0.484375, 478, 5),
                (29, 0.52734375, 491, 6),
                (29, 0.484375, 521, 6),
                (20, 0.106689453125, 838, 11)],
                [(99, 0.56982421875, 285, 4),
                (99, 0.51953125, 435, 8),
                (99, 0.48828125, 690, 12),
                (72, 0.352294921875, 1917, 16),
                (100, 0.414794921875, 1061, 18),
                (127, 0.3701171875, 973, 18)]]


In [12]:
import pandas as pd
data = pd.read_csv('../Data/ai-mathematical-olympiad-prize/train.csv')

In [13]:
ys = data.answer.tolist()

In [14]:
ys

[52, 250, 702, 800, 211, 199, 185, 320, 480, 199]

In [40]:
for i in range(1,200):
    correct = 0
    for path,y in zip(total_paths,ys):
        yhat = max(path,key=lambda x:x[1]+x[3]**2*0.00001*i)[0]
        correct += y == yhat
    print(i,correct)
# max(path,key=lambda x:x[1]+x[3]**2*0.00001*107)[0]

1 2
2 2
3 2
4 2
5 2
6 2
7 2
8 2
9 2
10 2
11 2
12 2
13 2
14 2
15 2
16 2
17 2
18 2
19 2
20 2
21 2
22 2
23 2
24 2
25 2
26 2
27 2
28 2
29 2
30 2
31 2
32 2
33 2
34 2
35 2
36 2
37 2
38 2
39 2
40 2
41 2
42 2
43 2
44 2
45 2
46 2
47 2
48 2
49 2
50 2
51 2
52 2
53 2
54 2
55 2
56 2
57 2
58 2
59 2
60 2
61 2
62 2
63 2
64 2
65 2
66 2
67 2
68 2
69 2
70 2
71 2
72 2
73 2
74 2
75 2
76 2
77 2
78 2
79 2
80 2
81 2
82 2
83 2
84 2
85 2
86 2
87 2
88 2
89 2
90 2
91 2
92 2
93 2
94 2
95 2
96 3
97 3
98 3
99 3
100 3
101 3
102 3
103 3
104 3
105 3
106 3
107 3
108 3
109 3
110 3
111 3
112 3
113 3
114 3
115 3
116 3
117 3
118 3
119 3
120 3
121 2
122 2
123 2
124 2
125 2
126 2
127 2
128 2
129 2
130 2
131 2
132 2
133 2
134 2
135 2
136 2
137 2
138 2
139 2
140 2
141 2
142 2
143 2
144 2
145 2
146 2
147 2
148 2
149 2
150 2
151 2
152 2
153 2
154 2
155 2
156 2
157 2
158 2
159 2
160 2
161 2
162 2
163 1
164 1
165 1
166 1
167 1
168 1
169 1
170 1
171 1
172 1
173 1
174 1
175 1
176 1
177 1
178 1
179 1
180 1
181 1
182 1
183 1
184 1
185 

In [31]:
correct = 0
for path,y in zip(total_paths,ys):
    # yhat = max(path,key=lambda x:x[1]+x[3]**2*0.0001*i)[0]
    # correct += y == yhat
    correct += y in [x[0] for x in path]

In [32]:
correct

3

In [ ]:
df['answer'] = final_answers

In [ ]:
df

In [ ]:
df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [ ]:
df[['id','answer']].head()